In [1]:
import moldesign as mdt
from moldesign import units as u
import ipywidgets as ipy

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med
# mdt.configure()

In [3]:
mdt.Atom


moldesign.molecules.atoms.Atom

In [4]:
mol = mdt.from_pdb('1yu8')

In [5]:
mol = mdt.Molecule([res for res in mol.residues if res.type == "protein"])

In [6]:
mdmol = mdt.assign_forcefield(mol)
atom = mdmol.atoms[13]
print "Done parameterizing"

Connecting to docker host at unix://var/run/docker.sock... done


Done parameterizing


In [7]:
rs = mdt.widgets.ResidueSelector(mdmol)
rs

In [8]:
canvas = mdmol.draw()

In [9]:
mdmol.set_energy_model(mdt.models.LAMMPSInteractive, affected_atom=mdmol.atoms[657])
mdmol.calculate()
mdmol.set_integrator(integrator=mdt.integrators.LAMMPSNvt, timestep=1.0*u.fs, 
                     temperature=300.0*u.kelvin, frame_interval=40.0*u.fs)

LAMMPS output is captured by PyLammps wrapper
LAMMPS Description

1013 atoms
1024 bonds
1862 angles
4484 dihedrals
0 impropers

14 atom types
60 bond types
140 angle types
163 dihedral types

-200 200 xlo xhi
-200 200 ylo yhi
-200 200 zlo zhi

Masses

1 14.01
2 1.008
3 12.01
4 1.008
5 1.008
6 12.01
7 16.0
8 1.008
9 16.0
10 1.008
11 1.008
12 1.008
13 1.008
14 32.06

Pair Coeffs

1 0.169999999918 3.24999852403 0.169999999918 3.24999852403
2 0.0157000000042 1.06907846172 0.0157000000042 1.06907846172
3 0.109399999916 3.39966950845 0.109399999916 3.39966950845
4 0.0157000000627 1.95997717901 0.0157000000627 1.95997717901
5 0.0157000000262 2.64953278726 0.0157000000262 2.64953278726
6 0.0860000001284 3.39966950794 0.0860000001284 3.39966950794
7 0.209999999842 2.95992190164 0.209999999842 2.95992190164
8 0.0157000000985 2.47135304264 0.0157000000985 2.47135304264
9 0.210400000249 3.06647338746 0.210400000249 3.06647338746
10 0 0.0 0 0.0
11 0.0150000000642 2.59964245874 0.0150000000642 2.599

In [ ]:
def apply_user_force(x, y, z):
    vec = [x, y, z]
    if all(v==0.0 for v in vec):
        return;
    vec = vec * u.angstrom
    mdmol.energy_model.apply_force(vec)
    mytraj = mdmol.run(200*u.fs)
    
    for step in mytraj:
        mdmol.energy_model.render_molecule(traj=step)
        canvas.set_positions(step.positions.value_in(u.angstrom))
#     for step in xrange(5):
#         mdmol.run(50 * u.fs)
#         mdmol.energy_model.render_molecule()
#         canvas.set_positions()
    
    mdmol.energy_model.finish_interaction()
    canvas.set_positions()
    
#     mdmol.set_energy_model(mdt.models.LAMMPSInteractive, affected_atom=mdmol.atoms[657])
#     mdmol.calculate()

In [11]:
canvas.set_positions()
canvas.stick()
canvas

Widget Javascript not detected.  It may not be installed or enabled properly.


In [12]:
from ipywidgets import interact, interactive, fixed
interact(apply_user_force, x=ipy.FloatSlider(min=-20.0, max=20.0, step=10.0, continuous_update=False),
            y=ipy.FloatSlider(min=-20.0, max=20.0, step=10.0, continuous_update=False),
            z=ipy.FloatSlider(min=-20.0, max=20.0, step=10.0, continuous_update=False))

[0.56855763313609475, 0.56855763313609475, -0.56855763313609475]
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 1000.0 fs to 1200.0 fs
